In [1]:
from elasticsearch import Elasticsearch

In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)
import os 
import json

es = Elasticsearch(
    os.getenv("ES_URL"), 
    basic_auth=(os.getenv("ES_USERAME"), os.getenv("ES_PASSWORD"))
)
print(os.getenv("ES_URL"))

search_str = "6825"

body_params = {
    "size": 50,
    "sort": [{"@timestamp": "desc"}],
    "query": {
        "bool": {
            "should": [
                {
                    "wildcard": {
                        "sessionId": { "value": "*{}*".format(search_str) }
                    }
                },
                {
                    "wildcard": {
                        "cartId": { "value": "*{}*".format(search_str) }
                    }
                }
            ]
        }
    }
}

res = es.search(index=os.getenv("ES_INDEX"), body=json.dumps(body_params))

print(res['hits']['hits'])

http://localhost:9200
[{'_index': 'test-logs', '_id': 'QsWxwJcBISrdcGFvmVDG', '_score': None, '_source': {'@timestamp': '2025-06-08T11:04:10Z', 'level': 'INFO', 'service': 'cart-service', 'message': 'Session started', 'cartId': 'CART33', 'sessionId': 'SESS6825'}, 'sort': [1749380650000]}, {'_index': 'test-logs', '_id': 'QcWxwJcBISrdcGFvmVCN', '_score': None, '_source': {'@timestamp': '2025-06-08T11:03:15Z', 'level': 'INFO', 'service': 'order-service', 'message': 'Order placed', 'cartId': 'CART33', 'sessionId': 'SESS06825', 'orderNum': 'ORD795'}, 'sort': [1749380595000]}, {'_index': 'test-logs', '_id': 'QMWxwJcBISrdcGFvmVBh', '_score': None, '_source': {'@timestamp': '2025-06-08T11:02:00Z', 'level': 'INFO', 'service': 'cart-service', 'message': 'Item added to cart', 'cartId': 'CART33', 'sessionId': 'SESS06825', 'item': 'mob002'}, 'sort': [1749380520000]}, {'_index': 'test-logs', '_id': 'P8WxwJcBISrdcGFvmVAr', '_score': None, '_source': {'@timestamp': '2025-06-08T11:01:05Z', 'level': 'IN

In [3]:
# examples = [
#     {
#         "question": "Show me the cart details for Cart123",
#         "query": '{ "query": { "match": { "cartId": "Cart123" } } }'
#     },
#     {
#         "question": "What are the orders placed between June 20 to June 25 2025?",
#         "query": '{ "query": { "bool": { "must": [ { "range": { "@timestamp": { "gte": "2025-06-20T00:00:00Z", "lte": "2025-06-25T23:59:59Z" } } }, { "exists": { "field": "orderNum" } } ] } } }'
#     }
# ]

examples = [
    {
        "question": "Show me the cart details for Cart123",
        "query": """{{ "query": {{ "match": {{ "cartId": "Cart123" }} }} }}"""
    },
    {
        "question": "What are the orders placed between June 20 to June 25 2025?",
        "query": """{{ "query": {{ "bool": {{ "must": [
            {{ "range": {{ "@timestamp": {{ "gte": "2025-06-20T00:00:00Z", "lte": "2025-06-25T23:59:59Z" }} }} }},
            {{ "exists": {{ "field": "orderNum" }} }}
        ] }} }} }}"""
    }
]

In [4]:
from langchain_core.prompts import PromptTemplate

# user_prompt = PromptTemplate.from_template("Question: {question} \n ES Query: {query}")
example_prompt = PromptTemplate(
    input_variables=["question", "query"],
    template="Question: {question}\nES Query: {query}"
)

In [5]:
# Test the prompt formatting
example_prompt.invoke(examples[0])

StringPromptValue(text='Question: Show me the cart details for Cart123\nES Query: {{ "query": {{ "match": {{ "cartId": "Cart123" }} }} }}')

In [6]:
from langchain_core.prompts import FewShotPromptTemplate 

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are an Elastisearch DSL generator",
    suffix="User input: {question}\n ES Query:",
    input_variables=["question"]
)
prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'Show me the cart details for Cart123', 'query': '{{ "query": {{ "match": {{ "cartId": "Cart123" }} }} }}'}, {'question': 'What are the orders placed between June 20 to June 25 2025?', 'query': '{{ "query": {{ "bool": {{ "must": [\n            {{ "range": {{ "@timestamp": {{ "gte": "2025-06-20T00:00:00Z", "lte": "2025-06-25T23:59:59Z" }} }} }},\n            {{ "exists": {{ "field": "orderNum" }} }}\n        ] }} }} }}'}], example_prompt=PromptTemplate(input_variables=['query', 'question'], input_types={}, partial_variables={}, template='Question: {question}\nES Query: {query}'), suffix='User input: {question}\n ES Query:', prefix='You are an Elastisearch DSL generator')

In [7]:
# Debug code
print("Example keys:")
for i, ex in enumerate(prompt.examples):
    print(f"Example {i}:", list(ex.keys()))

Example keys:
Example 0: ['question', 'query']
Example 1: ['question', 'query']


In [ ]:
final_prompt = prompt.format(
    question="Show me the details of cart id CART-A1F7"
)
final_prompt